In [1]:
!pip install selenium

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import random
import time
import pandas as pd
from datetime import datetime
import re
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from openpyxl.utils import escape
from openpyxl.cell.cell import ILLEGAL_CHARACTERS_RE
from urllib.parse import urljoin
import requests
from selenium.common.exceptions import WebDriverException
from urllib3.exceptions import MaxRetryError, NewConnectionError
import traceback


In [3]:
import os

In [4]:
from selenium.webdriver.common.action_chains import ActionChains

In [5]:
chrome_driver_path = "D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"

<>:1: SyntaxWarning: invalid escape sequence '\c'
<>:1: SyntaxWarning: invalid escape sequence '\c'
C:\Users\nhatk\AppData\Local\Temp\ipykernel_32252\211932188.py:1: SyntaxWarning: invalid escape sequence '\c'
  chrome_driver_path = "D:\chromedriver-win64\chromedriver-win64\chromedriver.exe"


In [ ]:
options = Options()
options.headless = True  # không hiển thị trình duyệt
options.add_argument("--disable-gpu")
options.add_argument("--blink-settings=imagesEnabled=false")  # tắt hình ảnh
options.add_argument("--disable-extensions")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/115.0 Safari/537.36")

In [7]:
# Khởi tạo driver
driver = webdriver.Chrome(service=Service(chrome_driver_path), options=options)

In [8]:
def clean_text(text):
    if isinstance(text, str):
        return ILLEGAL_CHARACTERS_RE.sub("", text)
    return text

In [15]:
driver.get("https://cafef.vn/du-lieu/vhm-2164146/vhm-thong-bao-thoi-gian-dia-diem-va-link-cung-cap-tai-lieu-hop-dhdcd-thuong-nien-2025.chn")

In [16]:
driver.title

'VHM: Thông báo thời gian, địa điểm và link cung cấp tài liệu họp ĐHĐCĐ thường niên 2025 - VHM: Thong bao thoi gian, dia diem va link cung cap tai lieu hop DHDCD thuong nien 2025 | Tin tức doanh nghiệp niêm yết | | CafeF.vn'

In [38]:
import random

In [9]:
# new
def get_official_statement(original_file, destination_file, destination_official_statement_folder):
    def init_driver():
        # options = webdriver.ChromeOptions()
        # options.add_argument('--headless')  # Tùy chọn nếu bạn không cần giao diện
        # return webdriver.Chrome(options=options)

        options = Options()
        options.headless = True  # không hiển thị trình duyệt
        options.add_argument("--disable-gpu")
        options.add_argument("--blink-settings=imagesEnabled=false")  # tắt hình ảnh
        options.add_argument("--disable-extensions")
        options.add_argument("--no-sandbox")
        options.add_argument("--disable-dev-shm-usage")
        options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/115.0 Safari/537.36")
        return webdriver.Chrome(options=options)

    driver = init_driver()  # Khởi tạo driver ban đầu

    df = pd.read_excel(original_file)
    df_news_official_statement = df[df['Content'].fillna('').str.strip() == ''].copy()
    df_news_official_statement['official_statement_pdf_name'] = ""
    print(f"Đã load df: {df_news_official_statement.shape[0]}")

    os.makedirs(destination_official_statement_folder, exist_ok=True)
    max_retries = 5

    for i, row in df_news_official_statement.iterrows():
        url = row['url']

        for attempt in range(max_retries):
            try:
                time.sleep(random.uniform(1, 5))
                print(f"\n📰 Đang xử lý bài báo thứ {i} (Lần thử {attempt+1}):\n{url}")
                driver.get(url)
                
                wait = WebDriverWait(driver, 10)
                pdf_element = wait.until(EC.presence_of_element_located((
                    By.XPATH,
                    "//div[@id='newscontent']//a[contains(@href, '.pdf')]"
                )))
                pdf_url = pdf_element.get_attribute('href')
                full_pdf_url = urljoin(url, pdf_url)

                df_news_official_statement.at[i, 'Content'] = full_pdf_url
                print(f"📎 Tìm thấy link PDF: {full_pdf_url}")

                pdf_filename = os.path.basename(full_pdf_url)
                pdf_path = os.path.join(destination_official_statement_folder, pdf_filename)

                response = requests.get(full_pdf_url, timeout=10)
                if response.status_code == 200:
                    with open(pdf_path, 'wb') as f:
                        f.write(response.content)
                    print(f"📁 Đã tải file PDF: {pdf_filename}")
                    df_news_official_statement.at[i, 'official_statement_pdf_name'] = pdf_filename
                else:
                    print(f"⚠️ Không thể tải PDF: {full_pdf_url} (mã trạng thái {response.status_code})")
                break

            except Exception as e:
                err_msg = str(e)
                print(f"❌ Lỗi lần {attempt+1} khi xử lý URL {url}: {err_msg}")

                # Nếu lỗi liên quan đến mất kết nối WebDriver
                if (
                    "Max retries exceeded" in err_msg or 
                    "Failed to establish a new connection" in err_msg or 
                    isinstance(e, WebDriverException)
                ):
                    print("🔌 Phát hiện lỗi kết nối với driver. Đang khởi động lại...")
                    try:
                        driver.quit()
                    except:
                        pass  # Đảm bảo không crash nếu driver đã bị đóng

                    time.sleep(2)
                    driver = init_driver()
                    continue

                if attempt < max_retries - 1:
                    print("🔁 Đang thử lại sau 2 giây...")
                    time.sleep(2)
                else:
                    print("🚫 Bỏ qua URL này sau nhiều lần thử.")
                    break

    try:
        driver.quit()
    except:
        pass

    with pd.ExcelWriter(destination_file, engine='openpyxl', mode='w') as writer:
        df_news_official_statement.to_excel(writer, sheet_name='Sheet1', index=False)
    print(f"\n✅ Đã lưu file Excel mới chứa nghị quyết: {destination_file}")

In [ ]:
original_file = r'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v1\content_news_cafef\train_data\ACB.xlsx'
destination_file = r'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\ACB\ACB.xlsx'
destination_official_statement_file = r'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\ACB\official_statement_pdf'
get_official_statement(original_file, destination_file, destination_official_statement_file)

Đã load df: 41

📰 Đang xử lý bài báo thứ 0 (Lần thử 1):
https://cafef.vn/du-lieu/ACB-2109424/acb-ong-nguyen-van-hoa-thoi-kiem-nhiem-chuc-giam-doc-tai-chinh-tu-112025.chn
❌ Lỗi lần 1 khi xử lý URL https://cafef.vn/du-lieu/ACB-2109424/acb-ong-nguyen-van-hoa-thoi-kiem-nhiem-chuc-giam-doc-tai-chinh-tu-112025.chn: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='newscontent']/div[3]/a"}
  (Session info: chrome=136.0.7103.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF60DBECF65+75717]
	GetHandleVerifier [0x00007FF60DBECFC0+75808]
	(No symbol) [0x00007FF60D9B8F9A]
	(No symbol) [0x00007FF60DA0F4C6]
	(No symbol) [0x00007FF60DA0F77C]
	(No symbol) [0x00007FF60DA62577]
	(No symbol) [0x00007FF60DA373BF]
	(No symbol) [0x00007FF60DA5F39C]
	(No symbol) [0x00007FF60DA37153]
	(No symbol) [0x00007FF60DA00421]
	(No symb

In [10]:
def get_official_stament_for_stocks(tickers):
    for ticker in tickers:
        original_file = rf'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v1\content_news_cafef\train_data\{ticker}.xlsx'
        destination_file = rf'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\{ticker}\{ticker}.xlsx'
        destination_official_statement_file = rf'D:\thacsi\TAILIEULUANVAN\code\PredictStock_TA_FA_SA\SA\data\v2\news_official_statement\train_data\{ticker}\official_statement_pdf'
        get_official_statement(original_file, destination_file, destination_official_statement_file) 

In [ ]:
tickers = ['ACB', 'BCM', 'BID','BVH','CTG','FPT','GAS','GVR','HDB','HPG','LPB','MBB','MSN','MWG','PLX','SAB','SHB','SSB','SSI','STB','TCB','TPB','VCB','VHM','VIB','VIC','VJC','VNM','VPB','VRE']
get_official_stament_for_stocks(tickers)

Đã load df: 571

📰 Đang xử lý bài báo thứ 1 (Lần thử 1):
https://cafef.vn/du-lieu/vic-1922904/vhm-tap-doan-vingroupctcp-da-mua-116745200-cp.chn
📎 Tìm thấy link PDF: https://cafef1.mediacdn.vn/download/291223/vhm-bao-cao-ket-qua-giao-dich-co-phieu-cua-to-chuc-co-lien-quan-den-nguoi-noi-bo-tap-doan-vingroup-ctcp-0.pdf
📁 Đã tải file PDF: vhm-bao-cao-ket-qua-giao-dich-co-phieu-cua-to-chuc-co-lien-quan-den-nguoi-noi-bo-tap-doan-vingroup-ctcp-0.pdf

📰 Đang xử lý bài báo thứ 2 (Lần thử 1):
https://cafef.vn/du-lieu/vic-1922917/vic-hdqt-phe-duyet-giao-dich-voi-ben-lien-quan-nam-2024.chn
📎 Tìm thấy link PDF: https://cafef1.mediacdn.vn/download/291223/vic-h-qt-phe-duyet-giao-dich-voi-ben-lien-quan-nam-2024-0.pdf
📁 Đã tải file PDF: vic-h-qt-phe-duyet-giao-dich-voi-ben-lien-quan-nam-2024-0.pdf

📰 Đang xử lý bài báo thứ 5 (Lần thử 1):
https://cafef.vn/du-lieu/vic-1918522/vic-bao-cao-ket-qua-giao-dich-co-phieu-esop-cua-nguoi-noi-bo-nguyen-thi-thu-hien-pham-thu-huong-pham-thuy-hang-pham-van-khuong.chn